# chap4-The-Cross-Entroy-Method

关于cross-entropy方法有以下两个特点：
* 简单，利用pytorch实现的代码行数不超过100行
* 这种方法具有很好的收敛性。对于简单的环境中，不需要复杂，多步骤的策略来学习和探索，在较短的周期内就可以获得频繁的反馈，cross-entropy方法通常效果比较好。

## The taxonomy of RL methods

cross-entropy 方法是免模型(model-free)和基于政策(policy-based)的方法。

* 免模型：意味着这种方法不会建立一个关于环境和奖励的模型，而是直接对现存的观测进行计算得到需要采取的动作。
* 基于策略：基于策略的方法可以近似看作是代理的策略，即代理在每一步应该执行什么操作，策略可以用操作的概念分布表示

所以cross-entropy方法是model-free，policy-based和on-policy的方法：
* 不需要构建环境模型，直接告诉代理如何在下一步采取动作
* 近似代理的策略
* 需要从环境中获取数据

## The cross-entropy method in pratice

在代理的活动周期，其对应的经历可以用一个周期(episode)表示，每一个周期(episode)是代理在环境中获得的观测，采取的动作以及从采取的动作获得奖励的集合。 

由于环境和代理采取动作的随机性，一些活动周期比另一些要好一些。cross-entropy方法的核心是将一些表现差的活动周期扔掉然后在表现好的活动周期上训练，所以，该方法的流程如下：
* 使用现有的模型和环境运行N次周期
* 计算每个周期的奖励，然后设定一个奖励边界，通常使用的是所有奖励分数的百分数，例如50%或者70%
* 丢弃所有奖励分数低于奖励边界的周期
* 在保留**精英**episode上进行训练，将观测作为输入，对应的动作作为想要的输出
* 从第一步开始重复直到结果令人满意

> 在上述步骤进行的过程中，神经网络学习如何重复进行动作来获得更大的奖励，然后将奖励分数变得越来越高。尽管这个算法很简单，但是它在基本的环境中运行效果很好，很容易实施，调整超参数的时候也非常稳定，让它成为一个理性的baseline。

## The cross-entropy method on CartPole

模型的核心是一个单隐藏层的神经网络，包含ReLU层和128个隐藏层神经元，该方法比较稳定并且收敛的速度非常快，所以设定的迭代次数较少

In [2]:
# import basic modules
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# set hyperparameters
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

### 128 hidden size neural

这个神经网络没有比较特殊的地方，它知识从环境中获取输入向量，然后输出我们可以采取的每一个动作的数字。神经网络的输出是表示动作的分布概率，所以最简单的处理方法就是在最后一层包含一个softmax非线性层。但是在处理神经网络的时候，不需要采取一个softmax层来增加训练过程中的数值稳定性。

这里可以直接采用`PyTorch`的类`nn.CrossEntropyLoss`来实现交叉熵损失计算，它包含了`softmax`和交叉熵计算，并且数字表达方式更加稳定。

In [4]:
# define net
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

### EpsideStep and Episode

定义了两个类来保存迭代过程的数据：
* EpisodeStep:表示代理在每一次迭代时采取的动作，保存了代理在环境中获取的观测以及采取的动作。最后使用精英`elite`的周期作为训练数据
* Episode:单个迭代周期存储在Episode中，对应的键值是该迭代周期总共的奖励分数

然后是在episode周期生成的训练用的batch：
在该函数中需要输入环境`env`，模型`net`以及需要生成多少步骤的`episode`。在每一次迭代的时候，将当前的观测转换为`PyTorch`向量`tensor`，然后将它传递给神经网络获取动作概率。以下是需要注意的几点：
* 在`PyTorch`中所有的`nn.module`实例期望获取的是一个数据实体的`batch`，当然对于这里定义的神经网络也是如此，所以需要将观测的数据转换为`1x4`的`tensor`
* 因为在神经网络的输出中没有采用非线性层，所以神经网络输出的是原始的动作分数，在神经网络的输出后再应用上`softmax`函数
* 神经网络和`softmax`层都返回了包含梯度的`tensor`，所以需要通过访问`torch.data`域来解包，然后将其转换为`NumPy`数组。该数组有相同的二维结构，我们只需要获取第一维的数据以获得动作概率。

现在有了动作的分布概率，我们可以通过`numpy`库的`random.choice()`来采样分布概率获取实际采取的动作。然后，将动作实施到环境中获取下一步的`observation`、`reward`以及周期是否结束的标志`is_done`

然后将奖励添加到当前的活动周期的总奖励分数中，`episode`同样用`(observation, action)`来表示。注意当前对应的`observation`是用来选择`action`的观测，而不是`action`采取之后环境变化之后的`observation`。虽然区别比较细微，但是还是需要注意这一点。

然后就是处理当前事件结束的情况。我们将完成的事件添加到`batch`，然后保存总的奖励分数和采取的步骤。然后重置总奖励累加变量，清除步骤保存列表，重新设置环境，再次开始。

> 另一个比较重要的点是在生成事件的同时，神经网络同时也在进行训练。在每一次迭代生成足够的事件`episode`的同时，我们也在利用梯度下降训练神经网络。所以当`yield`返回生成的数据的时候，神经网络会变得不同，朝着期望的方向变得稍微更好一些

In [5]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [6]:
# generate batches with episodes
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

### filter batch

这个函数是`cross-entropy`方法的核心，从给定一批的事件和百分数中计算奖励边界，用来筛选用来训练的精英事件。使用`np.percentile`函数获取奖励边界，从一系列的值中和希望的百分数中，计算百分位数。然后计算一个平均奖励分数，这个数只是用来观察。

然后对于该批次的每个事件，检测总的奖励分数是否高于奖励边界，如果高于就将其观测和动作添加到训练集中。

最后需要将选择出来的精英奖励和动作转换为`tensor`

In [7]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean


### Main

然后将所有的代码添加在一起，主要是包含训练迭代的训练

在这部分代码的一开始，创建了所有需要的对象：环境模型、神经网络、目标函数、优化器以及用于观察训练效果的`Tensorboard.SummaryWriter`

在训练过程中，迭代生成批数据，然后使用`filter_batch`函数过滤得到`elite`事件。其中的`reward_b`和`reward_m`是用来过滤的奖励边界和观测的奖励均值。之后将神经网络的梯度归零，将观测输入神经网络，获得其对应的动作分数。该动作分数传给目标函数，该目标函数在神经网络输出和代理采取的动作中计算cross-entropy损失。这个思想就是对神经网络强化学习，让其学习这些精英的动作来获得更高的分数。然后我们在交叉熵损失中计算梯度并让优化器调整神经网络。

> 可以看到，上述的方法中神经网络学习的是如何在环境中获得的观测中采取动作，不需要和环境进行任何其他的交互。所以这种方法的实施不太依赖于环境的细节。这就是强化学习模型的魅力。

In [8]:
if __name__ == "__main__":
    env = gym.make("CartPole-v0")
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.01)
    writer = SummaryWriter(comment="-cartpole")
    
    for iter_no, batch in enumerate(iterate_batches(
            env, net, BATCH_SIZE)):
        obs_v, acts_v, reward_b, reward_m = \
            filter_batch(batch, PERCENTILE)
        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
            iter_no, loss_v.item(), reward_m, reward_b))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_bound", reward_b, iter_no)
        writer.add_scalar("reward_mean", reward_m, iter_no)
        if reward_m > 199:
            print("Solved!")
            break
    writer.close()

C:\Users\Administrator\.conda\envs\rlbook\lib\site-packages\gym\envs\registration.py:506: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  f"The environment {path} is out of date. You should consider "


0: loss=0.697, reward_mean=22.3, rw_bound=26.0
1: loss=0.674, reward_mean=28.1, rw_bound=30.0
2: loss=0.661, reward_mean=31.8, rw_bound=37.5
3: loss=0.660, reward_mean=36.0, rw_bound=35.0
4: loss=0.632, reward_mean=39.3, rw_bound=43.5
5: loss=0.612, reward_mean=44.5, rw_bound=49.5
6: loss=0.612, reward_mean=45.4, rw_bound=57.5
7: loss=0.610, reward_mean=49.8, rw_bound=64.5
8: loss=0.604, reward_mean=45.0, rw_bound=50.0
9: loss=0.573, reward_mean=42.8, rw_bound=55.0
10: loss=0.587, reward_mean=66.2, rw_bound=65.0
11: loss=0.593, reward_mean=66.1, rw_bound=88.5
12: loss=0.570, reward_mean=69.5, rw_bound=86.0
13: loss=0.558, reward_mean=75.4, rw_bound=87.5
14: loss=0.577, reward_mean=63.2, rw_bound=68.5
15: loss=0.542, reward_mean=71.8, rw_bound=82.0
16: loss=0.554, reward_mean=78.0, rw_bound=86.0
17: loss=0.561, reward_mean=82.6, rw_bound=101.0
18: loss=0.569, reward_mean=55.1, rw_bound=60.5
19: loss=0.553, reward_mean=79.1, rw_bound=83.0
20: loss=0.551, reward_mean=82.9, rw_bound=94.0
2

In [13]:
# test
import time
env = gym.make("CartPole-v0")
env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs = env.reset()
# env.render()
sm = nn.Softmax(dim=1)
total_reward = 0.0
while True:
    obs_v = torch.FloatTensor([obs])
    act_probs_v = sm(net(obs_v))
    act_probs = act_probs_v.data.numpy()[0]
    action = np.random.choice(len(act_probs), p=act_probs)
    next_obs, reward, is_done, _ = env.step(action)
    total_reward += reward
    # time.sleep(1)
    if is_done:
        print(total_reward)
        break
    obs = next_obs
# env.close()

200.0


In [14]:
env.close()

## The cross-entropy method on FrozenLake

这个环境是`FrozenLake`，是一个所谓的网格世界分类，你的`agent`生活在一个4x4大小的网格中，只能朝着四个方向进行移动，上、下、左和右。`agent`的起点是左上角，目标是达到右底部。在某些特定的网格中存在陷阱，如果你掉入这些陷阱的话，该次事件结束，并且奖励为零。如果`agent`到达目标网格，获得1.0的奖励分数并且事件结束

为了让它变得复杂一点，可以定义这个世界的环境比较滑，毕竟是一个冰雪世界。所以`agent`采取的动作并不是和期望的一致-它有33%的概率可能会滑向右边或者左边。例如，如果你想要代理移动到左边，有33%的概率确实会移动到左边，另外33%的概率可能会在网格之前停下，最后33%概率可能会在网格之后停下。这样使得过程变得稍微艰难一些。

环境的观测空间是离散的，也就是包含0到15的数字，即当前在网格中的位置。动作空间同样是离散的，但是它可以从0到3。神经网络期望的是数值向量，我们可以采用传统的one-hot编码离散的输入。

但是直接采用`cross-entropy`方法会导致无法收敛，主要是因为：
* 对于训练来说，事件执行的步数需要是有限的，并且恰到好处的短
* 事件的总的奖励需要有足够的辨识度可以分辨好事件和坏事件
* 没有立即表示`agent`是否成功或者失败

对于本问题的解决方法，有以下几个：
* 更多的数据：对于`CartPole`环境，每次迭代16个事件已经足够训练了，但是对于`FrozenLake`需要至少100次才能得到成功的事件
* 对于奖励设置一个衰减因子：让总的奖励分数和事件的长度相关，对于总的奖励分数设置一个衰减因子从0.9到0.95。对于短的事件有较高的奖励分数，这样增加了奖励分布的变化情况，避免出现无法分开精英事件的情况。
* 减少学习率：让神经网络有更多的时间来平均更多的训练样本
* 保留精英事件一段事件：在之前的训练过程中，从环境中采样好的事件用以训练，然后就将其丢弃了。在`FrozenLake`中，成功的事件比较少见，所以可以考虑将其保存用以多次迭代训练
* 更长的训练时间：因为成功时间的稀疏性，以及动作的随机性，神经网络在特定的情况下很难决定采取何种行动。为了达到50%的准确率，至少需要5k次迭代训练



In [15]:
e = gym.make("FrozenLake-v1")

In [16]:
e.observation_space

Discrete(16)

In [17]:
e.action_space

Discrete(4)

In [18]:
e.render()


SFFF
FHFH
FFFH
HFFG


In [23]:
# import basic modules
import random
import gym, gym.spaces
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [21]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space,
                          gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs


def filter_batch(batch, percentile):
    filter_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(filter_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation,
                                 example.steps))
            train_act.extend(map(lambda step: step.action,
                                 example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound

In [ ]:
if __name__ == "__main__":
    random.seed(12345)
    env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1"))
    # env = gym.wrappers.Monitor(env, directory="mon", force=True)
    obs_size = env.observation_space.shape[0]
    n_actions = env.action_space.n

    net = Net(obs_size, HIDDEN_SIZE, n_actions)
    objective = nn.CrossEntropyLoss()
    optimizer = optim.Adam(params=net.parameters(), lr=0.001)
    writer = SummaryWriter(comment="-frozenlake-tweaked")

    full_batch = []
    for iter_no, batch in enumerate(iterate_batches(
            env, net, BATCH_SIZE)):
        reward_mean = float(np.mean(list(map(
            lambda s: s.reward, batch))))
        full_batch, obs, acts, reward_bound = \
            filter_batch(full_batch + batch, PERCENTILE)
        if not full_batch:
            continue
        obs_v = torch.FloatTensor(obs)
        acts_v = torch.LongTensor(acts)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()
        action_scores_v = net(obs_v)
        loss_v = objective(action_scores_v, acts_v)
        loss_v.backward()
        optimizer.step()
        print("%d: loss=%.3f, rw_mean=%.3f, "
              "rw_bound=%.3f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean,
            reward_bound, len(full_batch)))
        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_mean", reward_mean, iter_no)
        writer.add_scalar("reward_bound", reward_bound, iter_no)
        if reward_mean > 0.8:
            print("Solved!")
            break
    writer.close()

## The theoratical background of the cross-entropy method

[cross-entropy method simple intro](https://people.smp.uq.edu.au/DirkKroese/ps/eormsCE.pdf)

